# PRÁCTICA DE TERREMOTOS UTILIZANDO CASSANDRA

En esta práctica se usará la base de datos NoSQL de Cassandra, para el caso de uso de modelación de una serie de consultas.

En el caso de los datos sísmicos, el volumen, la variedad y, especialmente, la
velocidad de generación de la información (nuevos registros de sensores, actualizaciones de
magnitudes, alertas de tsunami, datos geoespaciales y de profundidad) hacen que un enfoque
relacional resulte poco eficiente para este caso de uso.

A continuación se observará el criterio de elección de Cassandra como base de datos NoSQL para modelar las consultas, así como el modelo de datos de las distintas tablas, inserción y actualización de datos, y la creación y ejecución de las consultas SQL propuestas en la actividad.

## 1. Ventajas y desventajas de Cassandra

Antes de empezar con ventajas y desventajas, se debe de analizar el teorema CAP, para saber de forma general si Cassandra es de utilidad en nuestro caso de uso de Terremotos.

### ===================
### Teorema CAP en Cassandra
### ===================

* ¿Que cumple simpre?:
    * Tolerancia a particiones (P): En un entorno real, Cassandra usa varios nodos a modo de backup siguiendo una topología de anillo. Por tanto, el sistema sigue funcionando aunque haya fallos o cortes de red entre nodos.
    * Disponibilidad (A): Siempre responde a las peticiones (lecturas/escrituras), aunque algunos datos puedan no estar totalmente sincronizados.

* ¿Que no cumple siempre?:
    * Consistencia (C): Se sacrifica parcialmente, ya que los datos pueden tardar un poco en propagarse entre nodos. Esto se conoce como consistencia eventual.

* ¿Realmente sirve Cassandra en este caso de uso?

    * Según el teorema CAP, Cassandra es adecuada para este caso porque en un sistema global y distribuido de datos sísmicos es más importante no perder datos y mantener el servicio disponible que tener consistencia instantánea en todas las réplicas.

### ======
### Ventajas
### ======

* Alta escalabilidad horizontal: permite añadir nodos fácilmente sin afectar la disponibilidad ni el rendimiento.

* Altísima velocidad de escritura: ideal para registrar datos de sensores y eventos sísmicos en tiempo real.

* Disponibilidad continua: sin un único punto de fallo gracias a su arquitectura distribuida y replicación entre nodos.

* Modelo flexible de datos: no se requiere esquema rígido, como pasa en SQL. Cassandra puede adaptarse fácilmente a nuevos tipos de datos, por ejemplo a nuevos parámetros geológicos que previamente no estaban definidos.

* Replicación geográfica: los datos se podrían replicarse entre regiones o continentes, útil para observatorios sísmicos internacionales.

* Lecturas rápidas por clave: muy eficiente para consultas que acceden por identificadores concretos (ID de evento, zona, etc.).

* Tolerancia a fallos: sigue funcionando incluso si algunos nodos o centros de datos quedan fuera de servicio.

### =========
### Desventajas
### =========

* No soporta transacciones complejas ni joins: En el momento que la consulta sea un poco más compleja de lo habitual y requiriese un acceso multitabla.

* Curva de aprendizaje: Requiere entender muy bien el modelo de datos y tener muy bien especificadas las consultas a resolver. Esto genera mucho debate a la hora de construir las tablas que ejecuten de manera óptima la query, si se quiere evitar el allow filtering.

* Consistencia eventual: Como se ha visto antes, puede haber pequeños retrasos en la sincronización entre réplicas.

## 2. CREACIÓN DE LAS TABLAS

### ==========================
### Celda 1: Inicialización y Conexión
### ==========================

In [33]:
import time
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra import OperationTimedOut

# --- Configuración ---
CASSANDRA_HOSTS = ['127.0.0.1'] 
PORT = 9042
KEYSPACE = 'seismic_data'

# 1. Conexión al Cluster con reintentos para dar tiempo al contenedor a inicializarse
cluster = None
session = None
RETRY_ATTEMPTS = 15
RETRY_DELAY_SEC = 10 

print(f"Intentando conectar a Cassandra en {CASSANDRA_HOSTS}:{PORT}...")

for i in range(RETRY_ATTEMPTS):
    try:
        # Me conecto desde el host de WSL
        cluster = Cluster(CASSANDRA_HOSTS, port=PORT) 
        session = cluster.connect()
        print(f"✅ Conexión exitosa al cluster en el intento {i+1}.")
        break
    except Exception as e:
        print(f"❌ Error de conexión (Intento {i+1}/{RETRY_ATTEMPTS}): {e}")
        if i < RETRY_ATTEMPTS - 1:
            print(f"Esperando {RETRY_DELAY_SEC} segundos...")
            time.sleep(RETRY_DELAY_SEC)
        else:
            print("🚨 Fallo al conectar después de varios intentos.")
            raise ConnectionError("No se pudo conectar al cluster de Cassandra.")


# 2. Establecer Keyspace y Verificar Versión
if session:
    # Crear Keyspace (se debe hacer en la sesión inicial)
    session.execute(f"""
        CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
        WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': '1'}}
    """)
    session.set_keyspace(KEYSPACE)
    print(f"🔑 KEYSAPCE '{KEYSPACE}' activo.")
    
    # Mostrar versión
    rows = session.execute("SELECT release_version FROM system.local;").one()
    print(f"Versión de Cassandra: {rows.release_version}")

Intentando conectar a Cassandra en ['127.0.0.1']:9042...
✅ Conexión exitosa al cluster en el intento 1.
🔑 KEYSAPCE 'seismic_data' activo.
Versión de Cassandra: 5.0.5


### =========================
### Creación de las 4 Tablas Optimizadas
### =========================

A continuación se procede a la creación de las 4 tablas, una para cada una de las queies propuestas en la actividad

Las consultas a realizar son las siguientes:
 - 5.a.i: Filtra terremotos por el año del evento mayor de 2015.
 - 5.a.ii: Filtra terremotos en el país donde ocurrió y que empiece por: “Japa...” para el caso de "Japan".
 - 5.b.i: Consulta por un país concreto donde se han producido terremotos, mostrando todos los terremotos con magnitud superior a 7.0.
 - 5.b.ii: Consulta por un país concreto donde se han producido terremotos, incluyendo sólo los que presenten riesgo potencial de tsunami.

In [ ]:
print("\nCreando las 4 Tablas optimizadas para las consultas...")

# ----------------------------------------------------------------------
# TABLA 1: earthquakes_by_year_range (Optimiza: 5.a.i -> eq_id y Año > 2015)
# Estrategia: Particionar por grupo de años (old vs. new) para escalar.
# Lógica PK: year_pk = 'False' (si < 2015) o 'True' (si >= 2015).
# Clustering Key 1: Permite el filtro de rango (event_year > 2015).
# ----------------------------------------------------------------------
session.execute("""
    CREATE TABLE IF NOT EXISTS earthquakes_by_year_range (
        year_pk boolean,      
        event_year int,
        eq_id text,
        
        event_time timestamp, 
        intensity_mmi decimal, 
        country text, 
        duration_sec int,
        
        PRIMARY KEY (year_pk, event_year, eq_id)
    ) WITH CLUSTERING ORDER BY (event_year ASC, eq_id ASC);
""")


# ----------------------------------------------------------------------
# TABLA 2: earthquakes_by_country_initial (Optimiza: 5.a.ii -> eq_id y País 'Japa...')
# Estrategia: Particionar por la inicial del país (A-Z) para acortar las particiones.
# Lógica PK: country_initial_pk = Primera letra del País (ej: 'J' para Japan).
# Clustering Key 1: Permite el filtro de rango (>= 'Japa' AND < 'Japb').
# ----------------------------------------------------------------------
session.execute("""
    CREATE TABLE IF NOT EXISTS earthquakes_by_country_starting_with_prefix (
        country_initial_pk text,
        country text,
        eq_id text,
        
        event_time timestamp, 
        intensity_mmi decimal,
        duration_sec int,
        
        PRIMARY KEY (country_initial_pk, country, eq_id)
    ) WITH CLUSTERING ORDER BY (country ASC, eq_id ASC);
""")


# ----------------------------------------------------------------------
# TABLA 3: earthquakes_by_country_and_magnitude (Optimiza: 5.b.i -> País y Magnitud > 7.0)
# Estrategia: La PK es el País, ya que la consulta es "por un país concreto".
# Clustering Key 1: Permite el filtro de rango (mw > 7.0).
# Clustering Key 2: Ordenar por evento más reciente.
# ----------------------------------------------------------------------
session.execute("""
    CREATE TABLE IF NOT EXISTS earthquakes_by_country_and_magnitude (
        country text,
        mw decimal,
        event_time timestamp,
        
        eq_id text,
        intensity_mmi decimal,
        duration_sec int,
        
        PRIMARY KEY (country, mw, event_time)
    ) WITH CLUSTERING ORDER BY (mw DESC, event_time DESC); 
""")


# ----------------------------------------------------------------------
# TABLA 4: tsunami_potential_by_country (Optimiza: 5.b.ii -> País y riesgo potencial=TRUE)
# Estrategia: Partición Compuesta (País, Tsunami Boolean) para acceder directamente a TRUE/FALSE.
# Clustering Key 1: Ordenar por evento más reciente.
# ----------------------------------------------------------------------
session.execute("""
    CREATE TABLE IF NOT EXISTS tsunami_potential_by_country (
        country text,
        tsunami_potential boolean,
        event_time timestamp,
        
        -- Datos Desnormalizados
        eq_id text,
        mw decimal,
        intensity_mmi decimal,
        
        PRIMARY KEY ((country, tsunami_potential), event_time) -- PK Compuesta
    ) WITH CLUSTERING ORDER BY (event_time DESC);
""")

print("Status=OK --> Las 4 Tablas han sido creadas/verificadas")


2. Creando las 4 Tablas optimizadas para las consultas...
✅ Las 4 Tablas han sido creadas/verificadas con el esquema de optimización revisado (SIN UDTs).


## 3. INSERCIÓN DE DATOS EN CASSANDRA

In [ ]:
import pycountry
import random

def random_country(country_set):
    countries = [c.name for c in pycountry.countries if c.name != "Czechia" and  c.name not in country_set]
    return random.choice(countries)


country_set = set(['Antarctica', 'Andorra', 'Chequia', 'Cameroon', 'Canada'])

while len(country_set) < 9:
    country_set.add(random_country(country_set=country_set))

top_9_countries = list(country_set)
print(top_9_countries)

def get_country():
    return random.choice(top_9_countries) if random.random() < 0.7 else 'China'

['Chequia', 'Barbados', 'Antarctica', 'Zambia', 'Canada', 'Andorra', 'Suriname', 'Cameroon', 'China', 'Romania']


In [4]:
def year_pk(year):
    return True if year >= 2015 else False

def country_initial(country):
    return country[0].upper() if isinstance(country, str) and country else "X"

In [ ]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("./data/earthquake_data_tsunami.csv")
df_sample = df.sample(n=100, random_state=42).reset_index(drop=True)  # 100 filas aleatorias del dataset

# === Insertar registros adaptados a las 4 tablas ===
for idx, row in df_sample.iterrows():
    eq_id = f"eq_{idx+1:03d}"
    event_year = int(row['Year']) if random.random() <= 0.5 else random.choice([2012, 2013, 2014])
    event_time = datetime(event_year, int(row['Month']), 1)
    intensity_mmi = float(row['mmi'])
    country = get_country()
    duration_sec = random.randint(10, 600)
    mw = float(row['magnitude'])
    tsunami = bool(row['tsunami'])

    # --- Tabla 1: earthquakes_by_year_range ---
    session.execute("""
        INSERT INTO earthquakes_by_year_range (year_pk, event_year, eq_id, event_time, intensity_mmi, country, duration_sec)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (year_pk(event_year), event_year, eq_id, event_time, intensity_mmi, country, duration_sec))

    # --- Tabla 2: earthquakes_by_country_starting_with_prefix ---
    country_initial_pk = country[0]
    session.execute("""
        INSERT INTO earthquakes_by_country_starting_with_prefix (country_initial_pk, country, eq_id, event_time, intensity_mmi, duration_sec)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (country_initial(country), country, eq_id, event_time, intensity_mmi, duration_sec))

    # --- Tabla 3: earthquakes_by_country_and_magnitude ---
    session.execute("""
        INSERT INTO earthquakes_by_country_and_magnitude (country, mw, event_time, eq_id, intensity_mmi, duration_sec)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (country, mw, event_time, eq_id, intensity_mmi, duration_sec))

    # --- Tabla 4: tsunami_potential_by_country ---
    session.execute("""
        INSERT INTO tsunami_potential_by_country (country, tsunami_potential, event_time, eq_id, mw, intensity_mmi)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (country, tsunami, event_time, eq_id, mw, intensity_mmi))

print("Status=OK --> 100 registros insertados en las 4 tablas adaptadas.")


✅ 100 registros insertados en las 4 tablas adaptadas.


## 4. UPDATE EN CASSANDRA

Elegimos la tabla *earthquakes_by_country_and_magnitude*, en la que vamos a actualizar un registro.

Tal y como nos pone el enunciado, tenemos que actualizar los registros de un país concreto, cambiando el nombre del país a letras mayúsculas. 

 - Ej: China → CHINA.

Usaremos "China" como país a actualizar, dado que en `get_country()` hemos puesto por defecto que retorne "China" un 30% de las veces, para estar bastante seguros que hay datos para actualizar. 

In [36]:

country_used_to_delete = 'China'


row = session.execute("""
    SELECT country, mw, event_time, eq_id, intensity_mmi, duration_sec
    FROM earthquakes_by_country_and_magnitude
    WHERE country = %s
""", (country_used_to_delete,)).one()

if not row:
    print(f"No se encontró ningún registro con country='{country_used_to_delete}'")
else:
    print(" -> Registro encontrado:", row)

    mw = row.mw
    event_time = row.event_time
    eq_id = row.eq_id
    intensity_mmi = row.intensity_mmi
    duration_sec = row.duration_sec

    # Insertamos el registro existente con el nuevo campo 'CHINA'
    insert_cql = """
        INSERT INTO earthquakes_by_country_and_magnitude 
        (country, mw, event_time, eq_id, intensity_mmi, duration_sec)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    session.execute(insert_cql, ('CHINA', mw, event_time, eq_id, intensity_mmi, duration_sec))
    print(f" -> Registro insertado con country='CHINA', eq_id={eq_id}")

    # Borramos el registro antiguo, ya que UPDATE no borra
    delete_cql = """
        DELETE FROM earthquakes_by_country_and_magnitude
        WHERE country=%s AND mw=%s AND event_time=%s
    """
    session.execute(delete_cql, (country_used_to_delete, mw, event_time))
    print(f" -> Registro eliminado para country='{country_used_to_delete}', eq_id={eq_id}")

    # Verificamos
    result = session.execute("""
        SELECT country, eq_id, mw, intensity_mmi, event_time
        FROM earthquakes_by_country_and_magnitude
        WHERE country = %s
    """, ('CHINA',))

    print("\n -> Registros verificados con country='CHINA':")
    for r in result:
        print(r)

 -> Registro encontrado: Row(country='China', mw=Decimal('7.5'), event_time=datetime.datetime(2007, 1, 1, 0, 0), eq_id='eq_087', intensity_mmi=Decimal('5.0'), duration_sec=215)
 -> Registro insertado con country='CHINA', eq_id=eq_087
 -> Registro eliminado para country='China', eq_id=eq_087

 -> Registros verificados con country='CHINA':
Row(country='CHINA', eq_id='eq_023', mw=Decimal('7.7'), intensity_mmi=Decimal('4.0'), event_time=datetime.datetime(2013, 2, 1, 0, 0))
Row(country='CHINA', eq_id='eq_087', mw=Decimal('7.5'), intensity_mmi=Decimal('5.0'), event_time=datetime.datetime(2007, 1, 1, 0, 0))


## 5. CONSULTAS CQL Y PRUEBAS

### ===================
### Definición de las Queries
### ===================

Query 5.a.i

In [6]:
def query_earthquake_by_year_range(session, year):  
    query = """
    SELECT * FROM earthquakes_by_year_range 
    WHERE year_pk = ?  
      AND event_year > ?
    """
    prepared = session.prepare(query)

    start_time = time.time()
    rows = session.execute(prepared, (year_pk(year), year))
    end_time = time.time()

    time_ms = (end_time - start_time) * 1000

    return rows, time_ms

Query 5.a.ii

In [7]:
def get_next_prefix(prefix):
    if not prefix:
        return None
    last_char = prefix[-1]
    next_char = chr(ord(last_char) + 1)
    return prefix[:-1] + next_char

def query_earthquake_by_country_prefix(session, country_prefix):
    c_i = country_initial(country_prefix)
    upper_bound = get_next_prefix(country_prefix)

    query = """
    SELECT * FROM earthquakes_by_country_starting_with_prefix 
    WHERE country_initial_pk = ? 
      AND country >= ? 
      AND country < ?
    """
    prepared = session.prepare(query)

    start_time = time.time()
    rows = session.execute(prepared, (c_i, country_prefix, upper_bound))
    end_time = time.time()

    time_ms = (end_time - start_time) * 1000

    return rows, time_ms

Query 5.b.i

In [8]:
from decimal import Decimal
def query_earthquakes_by_magnitude(session, country, min_magnitude):
    query = """
    SELECT * FROM earthquakes_by_country_and_magnitude 
    WHERE country = ? 
      AND mw > ?
    """
    prepared = session.prepare(query)

    start_time = time.time()
    rows = session.execute(prepared, (country, Decimal(str(min_magnitude))))
    end_time = time.time()

    time_ms = (end_time - start_time) * 1000

    return rows, time_ms

Query 5.b.ii

In [17]:
def query_tsunami_potential_by_country(session, country_name):
    
    query = """
    SELECT * FROM tsunami_potential_by_country 
    WHERE country = ? 
      AND tsunami_potential = ?
    """
    prepared = session.prepare(query)

    start_time = time.time()
    rows = session.execute(prepared, (country_name, True))
    end_time = time.time()

    time_ms = (end_time - start_time) * 1000

    return rows, time_ms

### ===================
### Testeo de las Queries
### ===================

#### 5.a.i

In [10]:
year = 2017
rows, elapsed_time = query_earthquake_by_year_range(session, year)

df = pd.DataFrame(list(rows))

if df.shape[0] > 0:
    frec_yearPk = df['year_pk'].value_counts().values
    print(df.head())
    print(f"\n-> Tiempo de ejecución: {elapsed_time} ms; Registros totales obtenidos: {df.shape[0]}")
    print(f'\n-> Frecuencia de registros >= 2015: {df[df['year_pk'] == True].shape[0]}')

else:
    if year < 2015:
        print(f"No hay ningún registro desde {year} hasta 2015")
    else:
        print(f"No hay ningún registro >= {year}")

   year_pk  event_year   eq_id   country  duration_sec event_time  \
0     True        2018  eq_055  Suriname           443 2018-08-01   
1     True        2018  eq_080  Cameroon           300 2018-02-01   
2     True        2019  eq_033   Romania           248 2019-01-01   
3     True        2019  eq_091  Barbados           481 2019-08-01   
4     True        2020  eq_084  Barbados           467 2020-06-01   

  intensity_mmi  
0           6.0  
1           7.0  
2           3.0  
3           6.0  
4           7.0  

-> Tiempo de ejecución: 9.198427200317383 ms; Registros totales obtenidos: 13

-> Frecuencia de registros >= 2015: 13


Se simula la query filtrando por un año mayor o igual a 2015. Cabe destacar que la lógica ha sido generalizada para soportar también años inferiores a 2015, buscando desde ese año hasta 2015 (Ej: 2013 hasta 2015). Se puede probar con otro año para validar varios casos y la tabla en cuestión.

Si el año es igual o superior a 2015 se observará un otuput con este formato:

    -> Tiempo de ejecución: 4.205226898193359 ms; Registros totales obtenidos: N

    -> Frecuencia de registros >= 2015: N

El número total de registros siempre será igual a la frecuencia de los registros mayores a 2015 --> N registros.

Si el año es inferior a 2015, se observará un output de este estilo:
    
    -> Tiempo de ejecución: 4.205226898193359 ms; Registros totales obtenidos: K

    -> Frecuencia de registros >= 2015: 0

La frecuencia de los registros mayores a 2015 será siempre 0, confirmándonos que las particiones actúan de manera correcta, mandando los registros con años >= 2015 a una partición y los inferiores a la otra partición, balanceando la carga.

#### 5.a.ii

In [15]:
rows, elapsed_time = query_earthquake_by_country_prefix(session, 'C')

df = pd.DataFrame(list(rows))
frecuencia_paises = df['country'].value_counts()
print(df.head())
print(f"\n-> Tiempo de ejecución: {elapsed_time} ms; Filas obtenidas: {df.shape[0]}")
print('\n-> Frecuencias de cada país:')
print(frecuencia_paises)

  country_initial_pk   country   eq_id  duration_sec event_time intensity_mmi
0                  C  Cameroon  eq_001           589 2012-08-01           6.0
1                  C  Cameroon  eq_006            39 2013-07-01           4.0
2                  C  Cameroon  eq_012           163 2002-07-01           7.0
3                  C  Cameroon  eq_013           320 2016-11-01           9.0
4                  C  Cameroon  eq_021           246 2014-05-01           4.0

-> Tiempo de ejecución: 6.857156753540039 ms; Filas obtenidas: 40

-> Frecuencias de cada país:
country
Cameroon    15
China       10
Chequia      8
Canada       7
Name: count, dtype: int64


Se puede observar que en cada partición se guardan las iniciales del país:

    -> Prefix = 'An' --> 'Andorra', 'Antartica'
    -> Prefix = 'Ch' --> 'Chequia', 'Chile', 'China' 
    -> Prefix = 'Chi' --> 'Chile', 'China' 

Se puede ver con esto, que las particiones por iniciales funcionan bien para balancear la carga y que además sea capaz de buscar paises por prefijos en una misma partición.

También se puede observar la actuación de la Clustering Key en el "df.head", dado que sacará lso registros ordenados alfabéticamente por Country.

#### 5.b.i

In [16]:
mw = 7.1
rows, elapsed_time = query_earthquakes_by_magnitude(session, 'China', mw)

df = pd.DataFrame(list(rows))

if df.shape[0] > 0:
    frecu_escala = df[df['mw'] >= 7.0].value_counts()
    print(df.head())
    print(f"\n-> Tiempo de ejecución: {elapsed_time} ms; Filas obtenidas: {df.shape[0]}")
    print(f'\n-> Frecuencia escalas superiores a 7.0:')
    print(frecu_escala)
else:
    print(f"No hay escala disponible a partir de {mw}")

  country   mw event_time  duration_sec   eq_id intensity_mmi
0   China  7.7 2013-02-01            30  eq_023           4.0
1   China  7.5 2007-01-01           215  eq_087           5.0

-> Tiempo de ejecución: 5.151271820068359 ms; Filas obtenidas: 2

-> Frecuencia escalas superiores a 7.0:
country  mw   event_time  duration_sec  eq_id   intensity_mmi
China    7.5  2007-01-01  215           eq_087  5.0              1
         7.7  2013-02-01  30            eq_023  4.0              1
Name: count, dtype: int64


En este filtro se puede observar que algunos de los países no contienen magnitudes superiores a 7.0

De acuerdo al dataset, la mayoría de las magnitudes no son superiores a 7, y al recolectar los datos de ese dataset, cuenta con esta implicación a tener en cuenta.

#### 5.b.ii

In [28]:
country = 'Cameroon'
rows, elapsed_time = query_tsunami_potential_by_country(session, country)

df = pd.DataFrame(list(rows))

if df.shape[0] > 0:
    print(df.head())
    print(f"\n-> Tiempo de ejecución: {elapsed_time} ms; Filas obtenidas: {df.shape[0]}")
    print(f'\n-> Frecuencia de riesgos potenciales de sunami en {country}: {df[df['tsunami_potential'] == True].shape[0]}')
else:
    print(f"{country} no tiene riesgo potencial de tsunami")

    country  tsunami_potential event_time   eq_id intensity_mmi   mw
0  Cameroon               True 2021-02-01  eq_049           8.0  7.1
1  Cameroon               True 2018-02-01  eq_080           7.0  7.2
2  Cameroon               True 2016-11-01  eq_013           9.0  7.8
3  Cameroon               True 2013-10-01  eq_051           6.0  7.3
4  Cameroon               True 2013-09-01  eq_094           9.0  6.9

-> Tiempo de ejecución: 4.462003707885742 ms; Filas obtenidas: 6

-> Frecuencia de riesgos potenciales de sunami en Cameroon: 6


Recordemos que en esta consulta, la partition key está compuesta con el país y el riesgo de tsunami, ya que así podemos buscar en la misma partición de una manera muy rápida, el país que tenga riesgo de tsunami, y a parte balanceando mucho más la carga dado que los registro de ese mismo país que no tienen rsiego de sunami van a ir a otra partición.

Como se puede evaluar en la frecuencia de riesgos de tsunami en el filtro, coincide con el total de registros obtenidos, lo que nos indica que la consulta es consistente, ya que solo queremos sacar los riesgos potenciales de dicho país.

In [37]:
cluster.shutdown()

Una vez se acabe todo el flujo de sesión, desconecta el cluster para no tener muchas sesiones abiertas.

Si se quiere volver a re ejecutar otra vez alguna celda concreta, tienes que volver a ejecutar la celda 1 de Inicialización y Conexión.